In [59]:
import pandas as pd
from dash import Dash, html, dcc
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

In [ ]:
df = pd.read_csv('dialise_fav_final2.csv', sep=';')
df.sample(5)

C:\Users\Marnie\AppData\Local\Temp\ipykernel_35100\2474414082.py:1: DtypeWarning: Columns (20,37,104,107,118,119) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('dialise_fav_final2.csv', sep=';')


,AP_MVM,AP_CONDIC,AP_GESTAO,AP_CODUNI,AP_AUTORIZ,AP_CMP,AP_PRIPAL,AP_VL_AP,AP_UFMUN,AP_TPUPS,...,fav_ACF_FLEBIT,fav_ACF_HEMATO,fav_ACF_VEIAVI,fav_ACF_PULSO,fav_ACF_VEIDIA,fav_ACF_ARTDIA,fav_ACF_FREMIT,fav_AP_NATJUR,fav_repeticao,espera_fav
363470,201807,PG,431330,2241161,4318206465705,2018-07-01,305010107.0,2602.07,431330,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
730449,202112,EP,430000,2257815,4321207942645,2021-12-01,305010107.0,1571.62,431350,5,...,N,N,N,N,0001,0001,1.0,3069.0,12.0,NaN
464267,201907,EP,430000,2244306,4319205035772,2019-06-01,305010107.0,400.00,431690,5,...,N,N,S,S,0,0,3.0,2062.0,7.0,NaN
499136,201911,PG,431490,2237601,4319209383863,2019-11-01,305010107.0,2604.05,431490,5,...,N,N,S,S,0003,0003,4.0,2011.0,4.0,NaN
305988,201712,EP,430000,2231069,4317201427165,2017-12-01,305010115.0,3549.94,430060,36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN


# Pergunta 1

In [48]:
# Tempo de espera - Grafico de Linha + média por ano
df_espera = pd.DataFrame()
df_espera['data_primeira_dialise']      = df['ATD_DTPDR']
df_espera['cronico']                    = df['Cronico']
df_espera['data_atendimento_dialitico'] = df['AP_CMP']
df_espera['obito']                      = df['AP_OBITO']
df_espera['data_atendimento_fav']       = df['fav_AP_CMP']
df_espera.sample(10)

,data_primeira_dialise,cronico,data_atendimento_dialitico,obito,data_atendimento_fav
540399,2019-04-12,repetido,2020-03-01,0,2019-12-01
49339,2004-05-06,repetido,2015-03-01,0,NaN
1009526,2023-03-14,repetido,2024-05-01,0,NaN
1027951,2024-06-03,agudo,2024-06-01,0,2024-09-01
548511,2020-04-13,agudo,2020-04-01,0,NaN
926440,2023-07-01,agudo,2023-09-01,0,NaN
614475,2019-02-03,repetido,2020-11-01,0,2019-06-01
428705,2014-12-19,repetido,2019-03-01,0,2015-01-01
901303,2022-08-11,repetido,2023-06-01,0,2022-10-01
307534,2017-01-16,repetido,2018-01-01,0,2017-03-01


In [49]:
# Para os casos de morte, podemos desconsiderar o tempo de espera (?)
df_espera_mortos_validos = df_espera[df_espera.obito == 1][df_espera.data_atendimento_fav.notna()]

# Remover todos os mortos
df_espera = df_espera[df_espera.obito != 1]

# Inserir apenas falecidos válidos
df_espera = pd.concat([df_espera, df_espera_mortos_validos])
df_espera.sample(10)

C:\Users\Marnie\AppData\Local\Temp\ipykernel_35100\2896329900.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_espera_mortos_validos = df_espera[df_espera.obito == 1][df_espera.data_atendimento_fav.notna()]


,data_primeira_dialise,cronico,data_atendimento_dialitico,obito,data_atendimento_fav
1037799,2023-12-18,repetido,2024-07-01,0,2024-07-01
1061810,2018-06-20,repetido,2024-08-01,0,2021-02-01
524701,2019-04-05,repetido,2020-01-01,0,2021-02-01
702263,2011-04-13,repetido,2021-09-01,0,2023-03-01
676667,2017-09-01,repetido,2021-06-01,0,2021-07-01
404212,2017-11-01,repetido,2018-12-01,0,2021-03-01
207497,2013-06-06,repetido,2016-12-01,0,2016-10-01
929535,2022-06-15,repetido,2023-09-01,0,2022-07-01
1094123,2021-06-26,repetido,2024-10-01,0,2019-10-01
404547,2016-11-17,repetido,2018-12-01,0,2016-05-01


In [50]:
# Priorizar remover repetidos NaN
df_espera = df_espera[df_espera.cronico != 'repetido']
df_espera.sample(5)

,data_primeira_dialise,cronico,data_atendimento_dialitico,obito,data_atendimento_fav
170361,2016-07-26,agudo,2016-07-01,0,2018-11-01
1114716,2024-10-16,agudo,2024-11-01,0,2024-12-01
657908,2021-03-26,agudo,2021-03-01,0,2020-06-01
1115454,2024-10-01,agudo,2024-11-01,1,2024-11-01
46841,2015-01-06,agudo,2015-03-01,0,2015-02-01


In [51]:
# Remover observações que não são agudos e que não tiveram atendimento FAV
# Para garantir, vamos remover todos que não tiveram data_primeira_dialise também
df_espera = df_espera[df_espera.cronico != 'agudo']
df_espera = df_espera[df_espera.data_atendimento_fav.notna()]
df_espera = df_espera[df_espera.data_primeira_dialise.notna()]
df_espera.sample(10)

,data_primeira_dialise,cronico,data_atendimento_dialitico,obito,data_atendimento_fav
37749,2014-10-11,cronico,2015-01-01,0,2014-11-01
415859,2018-10-26,cronico,2019-01-01,0,2018-09-01
1132206,2022-01-31,cronico,2024-12-01,0,2024-11-01
888952,2023-02-09,cronico,2023-05-01,0,2023-04-01
1107562,2004-12-22,cronico,2024-11-01,0,2024-12-01
1138697,2018-04-14,cronico,2024-12-01,0,2024-11-01
1114510,2007-06-29,cronico,2024-11-01,0,2024-11-01
191916,2016-05-25,cronico,2016-10-01,0,2015-10-01
97984,2015-03-15,cronico,2015-10-01,0,2015-11-01
338110,2018-01-19,cronico,2018-04-01,0,2018-05-01


In [52]:
# Calcular tempo de espera
df_espera['data_atendimento_fav'] = pd.to_datetime(df_espera['data_atendimento_fav'])
df_espera['data_primeira_dialise'] = pd.to_datetime(df_espera['data_primeira_dialise'])

# Calcular a diferença
df_espera['tempo_entre_fav_e_primeira_dialise'] = (
    df_espera['data_atendimento_fav'] - df_espera['data_primeira_dialise']
).dt.days  # em dias

df_espera['tempo_anos'] = df_espera['tempo_entre_fav_e_primeira_dialise'] / 365.25

# Remover esperas negativas
df_espera = df_espera[df_espera['tempo_entre_fav_e_primeira_dialise'] > 0]
df_espera.sample(10)

,data_primeira_dialise,cronico,data_atendimento_dialitico,obito,data_atendimento_fav,tempo_entre_fav_e_primeira_dialise,tempo_anos
422408,2018-11-16,cronico,2019-02-01,0,2020-08-01,624,1.708419
1107554,2004-12-22,cronico,2024-11-01,0,2024-11-01,7254,19.860370
645626,2019-07-22,cronico,2021-03-01,0,2022-07-01,1075,2.943190
650708,2020-12-31,cronico,2021-03-01,0,2021-05-01,121,0.331280
1109033,2022-03-16,cronico,2024-11-01,0,2024-11-01,961,2.631075
61578,2005-02-14,cronico,2015-05-01,0,2015-07-01,3789,10.373717
490918,2019-07-02,cronico,2019-10-01,0,2021-03-01,608,1.664613
202051,2016-09-22,cronico,2016-12-01,0,2016-12-01,70,0.191650
1111523,2022-03-07,cronico,2024-11-01,0,2024-11-01,970,2.655715
1116058,2018-04-24,cronico,2024-11-01,1,2024-12-01,2413,6.606434


In [53]:
# Manter dados apenas a partir de janeiro de 2014 devido criação do dataset

df_espera['data_atendimento_dialitico'] = pd.to_datetime(df_espera['data_atendimento_dialitico'])
df_espera['data_atendimento_fav'] = pd.to_datetime(df_espera['data_atendimento_fav'])
df_espera['data_primeira_dialise'] = pd.to_datetime(df_espera['data_primeira_dialise'])

df_espera = df_espera[
        (df_espera['data_atendimento_dialitico'] >= '2014-01-01') &
        (df_espera['data_atendimento_fav'] >= '2014-01-01') &
        (df_espera['data_primeira_dialise'] >= '2014-01-01') ]

In [65]:
# Grafico de tempo de espera
max_tempo = df_espera['tempo_entre_fav_e_primeira_dialise'].max()
bins = list(range(0, int(max_tempo) + 10, 10))

# Plotar o histograma
fig1_p1 = px.histogram(
    df_espera,
    x='tempo_entre_fav_e_primeira_dialise',
    nbins=len(bins),
    title='Distribuição do Tempo de Espera para atendimento no FAV',
    labels={'tempo_entre_fav_e_primeira_dialise': 'Tempo de espera (dias)', 'count': 'Quantidade de pessoas'},
)

# Personalizar layout
fig1_p1.update_layout(
    xaxis_title='Tempo de espera (dias)',
    yaxis_title='Quantidade de pessoas',
    bargap=0.05,
    template='plotly_white'
)

fig1_p1.show()


In [66]:
df_agrupado = df_espera.groupby('ano_primeira_dialise')['tempo_entre_fav_e_primeira_dialise'].mean().reset_index()

fig2_p1 = px.bar(
    df_agrupado,
    x='ano_primeira_dialise',
    y='tempo_entre_fav_e_primeira_dialise',
    text='tempo_entre_fav_e_primeira_dialise',
    labels={
        'ano_primeira_dialise': 'Ano da Primeira Diálise',
        'tempo_entre_fav_e_primeira_dialise': 'Tempo médio de espera (dias)'
    },
    title='Tempo Médio de Espera por Ano da Primeira Diálise até a FAV',
    color='tempo_entre_fav_e_primeira_dialise',
    color_continuous_scale='Blues'
)

# Personalizar layout
fig2_p1.update_traces(texttemplate='%{text:.0f} dias', textposition='outside')

fig2_p1.update_layout(
    xaxis=dict(tickmode='linear'),
    yaxis_title='Tempo médio de espera (dias)',
    xaxis_title='Ano da primeira diálise',
    uniformtext_minsize=8,
    uniformtext_mode='hide',
    plot_bgcolor='white',
    width=900,
    height=500
)

fig2_p1.show()


# Pergunta 2

In [73]:
# Garantir que a coluna data_primeira_dialise está em datetime
df_espera['data_primeira_dialise'] = pd.to_datetime(df_espera['data_primeira_dialise'])

# Criar coluna de ano-mês
df_espera['ano_mes'] = df_espera['data_primeira_dialise'].dt.to_period('M').astype(str)

# Agrupar por ano-mês
df_mes = df_espera.groupby('ano_mes')['tempo_entre_fav_e_primeira_dialise'].mean().reset_index()

# Converter ano-mês para datetime (para plotar no eixo)
df_mes['ano_mes_dt'] = pd.to_datetime(df_mes['ano_mes'])

# === Gráfico de Linha (sem pontos) ===
fig1_p2 = go.Figure()

fig1_p2.add_trace(go.Scatter(
    x=df_mes['ano_mes_dt'],
    y=df_mes['tempo_entre_fav_e_primeira_dialise'],
    mode='lines',  # <-- Somente linha, sem pontos
    name='Tempo médio de espera',
    line=dict(color='blue', width=2)
))

# === Tarja vermelha para COVID-19 (março de 2020 até dezembro de 2021) ===
fig1_p2.add_vrect(
    x0="2020-03-01", x1="2021-12-31",
    fillcolor="red", opacity=0.2,
    layer="below", line_width=0,
    annotation_text="COVID-19", annotation_position="top left"
)

# === Layout ===
fig1_p2.update_layout(
    title='Média do Tempo de Espera por Mês',
    xaxis_title='Ano',
    yaxis_title='Tempo médio de espera (dias)',
    template='plotly_white',
    width=1000,
    height=500,
    xaxis=dict(
        tickformat='%Y',  # Mostra apenas o ano no eixo X
        dtick="M12",      # Um tick por ano
        showgrid=True
    )
)

fig1_p2.show()



# DASH Dashboard

In [74]:
# === App Dash ===
app = Dash(__name__)

app.layout = html.Div(children=[
    html.H1(children='Dashboard - Tempo de Espera na FAV'),

    html.Div(children='''
        Análise dos tempos de espera entre a primeira diálise e o atendimento na FAV.
    '''),

    dcc.Graph(
        id='bar-plot',
        figure=fig1_p1
    ),
    dcc.Graph(
        id='histogram',
        figure=fig2_p1
    ),

    dcc.Graph(
    id='line-plot',
    figure=fig1_p2
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)